# Chapter7. 합성곱 신경망(CNN)

## 7.1 전체 구조

* CNN
    * 합성곱 계층(convolutional layer)
    * 풀링 계층(pooling layer)
    * 완전 연결 계층(fully-connected layer; Affine layer)

<img src="./images/7-1.png" width="400" height="400"/>
[7-1]

<img src="./images/7-2.png" width="400" height="400"/>
[7-2]

## 7.2 합성곱 계층

* 패딩(padding)
* 스트라이드(stride)

### 7.2.1 완전연결 계층의 문제점

* 문제점 : 데이터의 형상이 무시된다
    * 이미지의 경우 세로,가로,채널(색상)으로 구성된 3차원 데이터이지만 완전연결 계층에서는 1차원으로 바꿔야한다

> 합성곱 계층의 장점 : 형상을 유지한다

* 특징 맵(feature map) : 합성곱 계층의 입출력 데이터
    * 입력 데이터 : 입력 특징 맵
    * 출력 데이터 : 출력 특징 맵

### 7.2.2 합성곱 연산

* 합성곱 연산 = 필터 연산
* 필터는 커널이라고도 한다

<img src="./images/7-3.png" width="400" height="400"/>
[7-1]

* 단일 곱센-누산 진행

<img src="./images/7-4.png" width="400" height="400"/>
[7-4]

* 완전연결 신경망에서 가중치와 편향은 존재
* CNN에서는 필터의 매개변수가 가중치에 해당
* 필터는? => 필터는 항상 1 x 1 형태이다

<img src="./images/7-5.png" width="500" height="500"/>
[7-5]

### 7.2.3 패딩

* 패딩 : 합성곱 연산 수행 전 입력 데이터 주변을 특정 값으로 채우는 것
* 패딩은 주로 출력 크기를 조정할 목적으로 사용한다

<img src="./images/7-6.png" width="400" height="400"/>
[7-6]

### 7.2.4 스트라이드

* 스트라이드(stride) : 필터를 적용하는 위치 간격

<img src="./images/7-7.png" width="400" height="400"/>
[7-7]

#### (패딩과 스트라이드에 따른 출력크기 수치화)

입력 크기가 $(H, W)$, 필터 크기가 $(FH, FW)$ 출력 크기가 $(OH, OW)$ 패딩이 $P$ 스트라이드가 $S$ 일 때

<img src="./images/7.1.png" width="200" height="200"/>
[7.1]

* +1 왼쪽 분수가 정수가 되게 해야한다

### 7.2.5 3차원 데이터의 합성곱 연산

<img src="./images/7-8.png" width="400" height="400"/>
[7-8]

(3차원)

<img src="./images/7-9.png" width="400" height="400"/>
[7-9]

* 3차원 합성곱 연산에서 입력 데이터의 채널 수와 필터의 채널 수가 같아야 함
* 필터 자체의 크기는 자유롭지만, 모든 필터가 같은 크기여야함

### 7.2.6 블록으로 생각하기

<img src="./images/7-10.png" width="400" height="400"/>
[7-10]

* 위 예에서 출력 데이터는 한 장의 특징 맵
* 다수의 채널을 내보내려면? => 필터(가중치)를 다수 사용하기
* 즉, 필터의 수도 고려해야 함

<img src="./images/7-11.png" width="400" height="400"/>
[7-11]

* 필터의 가중치 데이터는 4차원 : (출력 채널 수, 입력 채널 수, 높이, 너비)

#### (+편향)

<img src="./images/7-12.png" width="400" height="400"/>
[7-12]

### 7.2.7 배치 처리

<img src="./images/7-13.png" width="500" height="500"/>
[7-13]

## 7.3 풀링 계층

* 풀링 : 세로 또는 가로 방향의 공간을 줄이는 연산

<img src="./images/7-14.png" width="500" height="500"/>
[7-14] 2 x 2 Max pooling을 스트라이드 2로 처리

* 풀링의 윈도우 크기와 스트라이드는 같은 값으로 설정하는 것이 일반적
* 이미지 인식 분야에서는 주로 최대 풀링을 사용

### 7.3.1 풀링 계층의 특징

* 학습해야 할 매개변수가 없다
* 채널 수가 변하지 않는다
* 입력의 변화에 영향을 적게 받는다(강건하다) [그림 7-16]

<img src="./images/7-16.png" width="400" height="400"/>
[7-16]

## 7.4 합성곱/풀링 계층 구현하기

### 7.4.1 4차원 배열

In [4]:
import numpy as np

In [5]:
x = np.random.rand(10, 1, 28,28)

In [6]:
x.shape

(10, 1, 28, 28)

In [ ]:
# numppy에서 아래 두 개는 같다

In [11]:
print(x[0][0].shape)

(28, 28)


In [12]:
print(x[0, 0].shape)

(28, 28)


### 7.4.2 im2col로 데이터 전개하기

* numpy는 for문을 사용하면 성능이 떨어진다
* for 대신 im2col이란느 함수를 사용한다(image to column)

* 입력 데이터는 행으로 배열

<img src="./images/7-17.png" width="400" height="400"/>
[7-17]
<img src="./images/7-18.png" width="400" height="400"/>
[7-18]

* 필터는 열로 배열

<img src="./images/7-19.png" width="500" height="500"/>
[7-19]

### 7.4.3 합성곱 계층 구현하기

In [15]:
import sys, os
sys.path.append(os.pardir)
from common.util import im2col

x1 = np.random.rand(1,3,7,7) 
col1 = im2col(x1, 5, 5, 1, 0)
print(col1.shape)

(9, 75)


In [16]:
x2 = np.random.rand(10, 3, 7, 7)
col2= im2col(x2 ,5, 5, 1, 0)
print(col2.shape)

(90, 75)


In [17]:
class Convolution:
    def __init__(self, W, b, stride=1, pad = 0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
    
    def forward(self, x):
        FN. C. FH. FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H+2*self.pad - FH) / self.stride)
        out_w = int(1+(W+2*self.pad - FW) / self.stride)
        
        col = im2col(x, FN, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T
        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0,3,1,2)
        
        return out

### 7.4.4 풀링 계층 구현하기

<img src="./images/7-21.png" width="400" height="400"/>
[7-21]
<img src="./images/7-22.png" width="400" height="400"/>
[7-22]

In [18]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1+(H-self.pool_h) / self.stride)
        out_w = int(1+(W-self.pool_w) / self.stride)
        
        col = im2col(x, self.pool_h, self.pool_w, self.stride , self.pad)
        col = col.reshape(-1, self.pool_h * self.pool_w)
        
        out = np.max(col, axis = 1)
        
        out = out.reshape(N, out_h, out_w, C).transpose(0,3,1,2)
        
        return out

## 7.5 CNN 구현하기

<img src="./images/7-23.png" width="400" height="400"/>
[7-23]

In [19]:
from collections import OrderedDict

In [29]:
from common.layers import Pooling, Relu, Affine, SoftmaxWithLoss

In [23]:
class SimpleNet:
    def __init__(self, input_dim = (1,28,28), conv_param ={'filter_num':30, 'filter_size' : 5, 'pad' :0, 'stride' :1},
                 hidden_size = 100, output_size=10, weight_init_std=0.01):
        
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size+2*filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))
        
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)
        
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'], conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
        
        self.lastlayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    
    def loss(self, x, t):
        y= self.predict(x)
        return self.lastlayer.forward(y,t)
    
    def gradient(self, x, t):
        self.loss(x,t)
        
        dout =1
        dout = self.lastlayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
        
        grads = {}
        grads['W1'] = self.layers['Conv1'].dW
        grads['b1'] = self.layers['Conv1'].db
        grads['W2'] = self.layers['Affine1'].db
        grads['b2'] = self.layers['Affine1'].db
        grads['W3'] = self.layers['Affine2'].db
        grads['b3'] = self.layers['Affine2'].db
        
        return grads
        

## 7.6 CNN 시각화하기

### 7.6.1 1번째 층의 가중치 시각화하기

<img src="./images/7-24.png" width="400" height="400"/>
[7-24]

* 학습 전 필터는 무작위로 초기화하기 때문에 흑백의 정도에 규칙성이 없다
* 학습 후 필터는 규칙성 있는 이미지가 되었다

<img src="./images/7-25.png" width="400" height="400"/>
[7-25]

* 에지 : 색상이 바뀐 경계선 / 블롭 : 국소적으로 덩어리진 영역 등을 필터가 반응

### 7.6.2 층 깊이에 따른 추출 정보 변화

* 딥러닝 시각화에 관한 연구에 따르면 계층이 깊어질수록 추출되는 정보, 즉 강하게 반응하는 뉴런은 더 추상화된다

* [7-26]은 일반 사물 인식을 수행한 8층 CNN : AlexNet

<img src="./images/7-26.png" width="500" height="500"/>
[7-26]

* 1층은 에지와 블롭, 3층은 텍스처, 5층은 사물의 일부, 마지막 완전열결 계층은 사물의 클래스에 뉴런이 반응한다
* 층이 깊어질수록 뉴런이 바응하는 대상이 단순한 모양에서 고급 정보롤 변화

## 7.7 대표적인 CNN

### 7.7.1 LeNet

<img src="./images/7-27.png" width="600" height="600"/>
[7-27]

* 합성곱 계층과 풀링 계층(정확히는 원소를 줄이기만하는 서브샘플링 계층) 반복하고 마지막에 완전연결 계층을 통해 결과 출력
* 현재 CNN과 비교하면 LaNet은 시그모이드 함수를 사용하는 데 반해 현재는 주로 ReLU를 사용
* LaNet은 서브샘플링을 하여 중간 데이터의 크기가 작아지지만 현재는 Max Pooling이 주류


### 7.7.2 AlexNet

<img src="./images/7-28.png" width="500" height="500"/>
[7-28]

* 활성화 함수를 ReLU로 이용
* LRN(Local Response Normalization)이라는 국소적 정규화 실시하는 계층 이용
* 드롭아웃 이용

#### (LaNet과 AlexNet을 보며...)

* 네트워크 구성에 큰 차이는 없지만 컴퓨터 기술의 진보가 있었음
* 대량의 데이터를 얻기 좋아졌고, 병렬 계산에 특화된 GPU가 보급되면서 대량의 연산 고속 수행 가능